In [49]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
import tensorflow as tf
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, activations, models, preprocessing, utils
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from nltk.tokenize import word_tokenize, sent_tokenize
from keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import nltk
import sqlite3
from nltk import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [50]:
json_file_path = "/content/classification_v2.json"

with open(json_file_path, 'r') as j:
     contents = json.loads(j.read())

In [51]:
def preprocess_and_tokenize_sentences(text):
    text = text.lower()
    # text = re.sub(r'[^a-zA-Z\s?.]', '', text)
    text = text.replace('?','')
    sentences = sent_tokenize(text)
    return sentences

In [52]:
tags_list=[]
sentences_list= []
for intents in contents['Intenst']:
    tags= intents['Tags']
    questions = intents['Questions']
    for question in questions:
        tags_list.append(tags)
        sentences = preprocess_and_tokenize_sentences(question)
        # print(sentences)
        sentences_list.extend(sentences)

In [53]:
classification_data = pd.DataFrame({'Sentence': sentences_list,'Tag': tags_list})
classification_data = classification_data.sample(frac=1, random_state=42).reset_index(drop=True)
classification_data.to_csv('classification.csv',index=False)

In [54]:
classification_data = pd.read_csv('/content/classification.csv')
classification_data.head()

,Sentence,Tag
0,what financial outlay is expected for the acqu...,Information tags
1,what would be the purchase cost including taxe...,Information tags
2,what would the xiaomi redmi note 11 4gb-128gb ...,Information tags
3,am i able to reserve items that aren't in stoc...,Non-Informations tags
4,could i see a demonstration of the product bef...,Non-Informations tags


In [55]:
chatbot_data= pd.read_csv('/content/data_test.csv')
chatbot_data.head()

,cauhoi,traloi
0,"hi, how are you doing?",i'm fine. how about yourself?
1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
2,i'm pretty good. thanks for asking.,no problem. so how have you been?
3,no problem. so how have you been?,i've been great. what about you?
4,i've been great. what about you?,i've been good. i'm in school right now.


In [56]:
labels_mapping = {
    'Information tags': 1,
    'Non-Informations tags': 0
}
classification_data['Tag'] = classification_data['Tag'].map(labels_mapping)

In [57]:
X = classification_data.Sentence
Y = classification_data.Tag
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
classification_data.sample(10)

,Sentence,Tag
157,are there any special deals when buying multip...,0
178,how much is an masstel hapi 30 4g,1
3,am i able to reserve items that aren't in stoc...,0
39,what amount would one pay for a honor x8a 8gb-...,1
193,what is the list price for the vivo y16 4gb-128gb,1
115,what is your policy for exchanging defective g...,0
75,could you suggest a product that would suit my...,0
132,please compute the total purchase amount for a...,1
64,what would the iphone 15 pro 128gb set me back,1
112,what steps do i follow to opt out of your mark...,0


In [58]:
all_texts = classification_data['Sentence'].tolist() + chatbot_data['cauhoi'].tolist() + chatbot_data['traloi'].tolist()

In [59]:
max_len = 10
tokenizer = Tokenizer(oov_token='OOV')
tokenizer.fit_on_texts(all_texts)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
print(X_train_sequences)
X_test_sequences = tokenizer.texts_to_sequences(X_test)
vocab_size = len(tokenizer.word_index) + 1
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_len, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len, padding='post')

[[146, 3, 2472, 4, 1128, 697, 185, 16, 6, 587, 2473, 1620, 291, 1123, 481], [33, 24, 704, 16, 4, 787, 482, 591, 254, 360, 291, 10, 862, 66, 2, 2590], [10, 11, 4, 45, 781, 16, 4, 643, 1649, 1465, 1466], [11, 7, 649, 5, 1461, 24, 374, 186, 97, 340, 7], [10, 1135, 9, 3, 484, 16, 2515, 861], [10, 15, 4, 2531, 16, 91, 1640], [9, 3, 14, 6, 860, 16, 2519, 1640], [48, 3, 2569, 4, 1039, 147, 186, 55, 1662, 16, 4, 305, 1034, 591, 161], [321, 2575, 4, 1128, 697, 862, 16, 6, 643, 1649, 1465, 1466, 1123, 867], [48, 3, 118, 27, 6, 1455, 16, 4, 247, 248, 1615, 254], [2, 58, 783, 6, 147, 1129, 16, 4, 644, 2476, 1124, 161], [48, 3, 1455, 4, 147, 16, 4, 247, 248, 1635, 254, 161], [146, 3, 2527, 6, 400, 8, 58, 2528, 24, 789], [48, 3, 127, 27, 62, 54, 11, 6, 1625, 860, 16, 648, 1656], [10, 211, 12, 2546, 549, 44, 38, 589], [35, 48, 2, 1142, 24, 1661, 861], [83, 9, 3, 791, 14, 38, 2580, 1664], [10, 11, 4, 1665, 16, 6, 247, 248, 2602, 254, 291], [10, 11, 4, 485, 147, 16, 4, 305, 1034, 545, 161], [35, 189, 2

In [60]:
questions = chatbot_data["cauhoi"].tolist()
answers = chatbot_data["traloi"].tolist()
answers_with_tags = list()
#lấy ra từ điển
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )
answers = list()

In [61]:
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

In [62]:
tokenized_questions = tokenizer.texts_to_sequences( questions )
print(tokenized_questions)
maxlen_questions = max( [len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions, maxlen = maxlen_questions, padding = 'post')
encoder_input_data = np.array(padded_questions)

tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )

tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , vocab_size )
decoder_output_data = np.array( onehot_answers )

[[1669, 35, 15, 3, 179], [32, 654, 35, 34, 595], [32, 164, 46, 255, 16, 485], [31, 175, 22, 35, 14, 3, 102], [97, 102, 107, 10, 34, 3], [97, 102, 46, 32, 17, 96, 69, 114], [10, 96, 9, 3, 39, 5], [2, 39, 5, 872], [9, 3, 23, 7, 54], [20, 89, 20, 6, 52, 153, 1670], [46, 873, 44, 96], [1481, 7, 45], [32, 179, 57, 35, 34, 3], [125, 129, 255], [22, 35, 14, 3, 102, 462], [97, 551, 102, 164, 46, 3], [32, 551, 17, 96, 69, 114], [309, 96, 9, 3, 1044], [32, 1671, 872, 69, 114], [15, 3, 1672, 7, 54], [20, 40, 169, 54, 15, 6, 86, 12, 65, 54], [46, 873, 44, 8], [35, 15, 3, 179, 123], [32, 179, 107, 10, 34, 3], [32, 655, 1673, 143, 3], [1674, 102, 46, 44, 3], [2, 170, 102, 129, 35, 34, 595], [2, 294, 96, 874], [119, 15, 3, 45, 5, 96], [32, 45, 5, 872], [35, 9, 3, 23, 7, 22, 463], [2, 23, 7, 22, 463, 24, 1675, 15, 164, 46, 69, 114], [20, 80, 794, 88, 123], [2, 53, 2, 41, 7, 339, 237], [20, 4, 552, 12, 795, 7, 282, 237, 123], [8, 58, 30, 875], [176, 380, 1676, 20, 876, 1145, 341], [2, 53, 7, 58, 30, 11

In [63]:
model_1= tf.keras.models.load_model('/content/classification.h5',compile=False)
model_2= tf.keras.models.load_model('/content/chatbot.h5',compile=False)

In [64]:
model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 10)]              0         
                                                                 
 embedding (Embedding)       (None, 10, 30)            84060     
                                                                 
 lstm (LSTM)                 (None, 16)                3008      
                                                                 
 FC1 (Dense)                 (None, 8)                 136       
                                                                 
 dropout (Dropout)           (None, 8)                 0         
                                                                 
 out_layer (Dense)           (None, 1)                 9         
                                                                 
 activation (Activation)     (None, 1)                 0     

In [65]:
model_2.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 19)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 21)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 19, 200)              560400    ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 21, 200)              560400    ['input_2[0][0]']             
                                                                                              

In [66]:
def make_inference_models():
  encoder_inputs = model_2.input[0] #input_1
  encoder_outputs, state_h_enc, state_c_enc = model_2.layers[4].output #lstm 1
  encoder_states = [state_h_enc, state_c_enc]
  encoder_model = Model(encoder_inputs, encoder_states)


  decoder_inputs= model_2.output[1] #input_2
  decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
  decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
  decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
  decoder_embedding = model_2.layers[3](decoder_inputs) #embedding_1


  decoder_lstm = model_2.layers[5] #lstm 2
  decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
  decoder_states = [state_h_dec, state_c_dec]

  decoder_dense = model_2.layers[6] #dense layer
  decoder_outputs = decoder_dense(decoder_outputs)
  decoder_model = Model(
          [decoder_inputs] + decoder_states_inputs,
          [decoder_outputs] + decoder_states)
  return encoder_model , decoder_model

def str_to_tokens( sentence : str ):

    words = sentence.lower().split()
    tokens_list = list()

    for word in words:
        try:
            tokens_list.append( tokenizer.word_index[ word ] )
        except:
            tokens_list.append(tokenizer.word_index['i dont know'])

    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

enc_model , dec_model = make_inference_models()

def chat_bot_lstm(cau_hoi):
    states_values = enc_model.predict( str_to_tokens( cau_hoi ) )
#     print(states_values)
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
        empty_target_seq = np.zeros( ( 1 , 1 ) )
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ]
    return ( decoded_translation[1:-4])

In [67]:
def prediction(user_input_padded,thershold=0.003):
    predict = model_1.predict(user_input_padded)
    print(user_input_padded)
    print(predict)
    if predict[0] >= thershold:
        return 'Information Tag'
    else:
        return 'Non-Information Tag'

In [68]:
def remove_padding(input_padded):
    unpadded_input = []
    for sequence in input_padded:
        sentence = [word for word in sequence if word != 0]  # Assuming 0 is the padding token
        # print(sentence)
        unpadded_input.append(sentence)
    return unpadded_input

# Reverse the tokenization
def reverse_tokenization(sequences, tokenizer):
    original_sentences = []
    for sequence in sequences:
        # print(sequence)
        original_sentence = tokenizer.sequences_to_texts([sequence])[0]
        # print(original_sentence)
        original_sentences.append(original_sentence)
    return original_sentences

In [69]:
def pos_tokenize(text):
  words = word_tokenize(text)
  tags = pos_tag(words)
  selected_words = [word for word, tag in tags if tag in ['NN', 'NNP','CD','JJ']]
  return selected_words

In [70]:
def classification_tags(predict):
  if predict == 'Information Tag':
      original_user_input = temp
      price_tags = [
    'cost', 'price', 'how much', 'sell', 'budget', 'quotation', 'amount',"lowest price","total"
    'market value', 'pricing information', 'price list', 'price range',"final cost","sale price"
    'purchase cost', 'price point', 'expense', 'valuation',"rate", "cost range","price estimation"
    'invoice','list price', 'top price', 'bottom price',"retail price","expenditure"
    'final price', 'bill',"sold","purchase","spend","pay","price tag","current price",'much'
]
      screen_resolution_tags = [
    'screen resolution', 'clarity', 'Full HD', 'Retina display', 'contrast level',
    'color reproduction', 'OLED display', 'LCD display', 'brightness setting',
    'glare-resistant screen', 'refresh rate', '4K resolution',
    'QHD display', 'IPS technology', 'AMOLED screen', 'pixel density',
    'display quality', 'picture quality', 'PPI', 'HD', 'visual sharpness',
    'display specs', 'image definition', 'aspect ratio', 'contrast ratio',
    'color accuracy', 'brightness levels', 'display type', 'screen clarity',
    'viewing experience', 'pixels','screen','size'
]
      phone_memory_keywords = [
    "Memory", "Storage", "GB", "TB", "Storage Space", "Capacity", "RAM",
    "Internal Memory", "Built-in Storage", "Phone Storage", "External Storage",
    "Expandable Storage", "Memory Card Slot", "microSD Support", "microSD Slot",
    "UFS", "Memory Upgrade", "Storage Expansion", "ROM", "System Memory",
    "User Available Memory", "Cache Memory", "Phone Memory", "Device Storage",
    "Onboard Storage", "Secure Storage", "Memory Card Capacity",
    "Dual SIM with Memory Card Slot", "LPDDR4", "LPDDR5",
    "Read-Only Memory", "Flash Memory", "eMMC", "NVMe",
    "USB OTG Support", "Memory Efficiency", "High-Speed Memory",
    "Memory Bandwidth", "Storage Speed", "Memory Card Compatibility"
]

      # information_tags = ['information']
      tokenize = pos_tokenize(original_user_input)
      # print(tokenize)
      for keyword in tokenize:
        if keyword in price_tags:
          filtered_elements = [element for element in tokenize if element not in price_tags]
          concatenated_string = ' '.join(filtered_elements)
          return 'price_tag', concatenated_string
        elif keyword in screen_resolution_tags:
          filtered_elements = [element for element in tokenize if element not in screen_resolution_tags]
          concatenated_string = ' '.join(filtered_elements)
          return 'resolution_tag', concatenated_string
        elif keyword in phone_memory_keywords:
          filtered_elements = [element for element in tokenize if element not in screen_resolution_tags]
          concatenated_string = ' '.join(filtered_elements)
          return 'memory_tag', concatenated_string
  else:
    original_user_input = temp
    return "Non-Information Tag",chat_bot_lstm(original_user_input)


In [77]:
user_input = input("Enter your question: ")
temp = user_input
# original_user_input = user_input
# Preprocess and tokenize user input
user_input = preprocess_and_tokenize_sentences(user_input)
user_input_sequences = tokenizer.texts_to_sequences(user_input)
user_input_padded = pad_sequences(user_input_sequences, maxlen=max_len, padding='post')
user_input_padded

Enter your question: see you again


array([[ 64,   3, 308,   0,   0,   0,   0,   0,   0,   0]], dtype=int32)

In [78]:
predict = prediction(user_input_padded)
predict

1/1 [==============================] - 0s 20ms/step
[[ 64   3 308   0   0   0   0   0   0   0]]
[[0.00025918]]


'Non-Information Tag'

In [79]:
classification_tags(predict)

1/1 [==============================] - 1s 1s/step


1/1 [==============================] - 0s 27ms/step


('Non-Information Tag', 'no ten of my coworkers got laid off too')